# Multi-Tool Agent with LlamaStack 0.3.0

This notebook demonstrates how to create an agent that can use **multiple tools** and let the LLM **automatically choose** the right tool based on the query.

## Tools Available:
1. **indian_bank_stock** - Fetch stock prices for Indian banks (HDFC, ICICI, SBI) via Yahoo Finance
2. **web_search** - Search the web for real-time information (gold prices, exchange rates, news) via Tavily

## Approach:
We use LlamaStack's **Chat Completions API** with OpenAI-compatible function calling. The LLM decides which tool to use, and we execute it client-side.

In [1]:
# Install dependencies
%pip install -q llama_stack_client==0.3.0 yfinance rich


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
import yfinance as yf
import rich
from llama_stack_client import LlamaStackClient

In [ ]:
# LlamaStack service URL (in-cluster)
LLAMASTACK_URL = "http://llama-stack-dist-service.competitor-analysis.svc.cluster.local:8321"

# For access from Notebooks external to the cluster, use the route URL instead
# LLAMASTACK_URL = "https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/"

# Tavily API key for web search
tavily_search_api_key = os.getenv('TAVILY_SEARCH_API_KEY', 'tvly-xxxxxxx')

# Initialize client with Tavily provider data
client = LlamaStackClient(
    base_url=LLAMASTACK_URL,
    provider_data={"tavily_search_api_key": tavily_search_api_key},
    timeout=300.0
)

# Get available model
models = client.models.list()
model_id = next(m.identifier for m in models if m.model_type == "llm")
print(f"Using model: {model_id}")

INFO:httpx:HTTP Request: GET https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/models "HTTP/1.1 200 OK"


Using model: vllm-inference/granite-3-3-8b-instruct


## Define Tool Schemas

We define both tools using OpenAI-compatible function schemas. The LLM will see these definitions and decide which tool to call based on the user's query.

In [4]:
# Tool definitions using OpenAI-compatible function schema
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "indian_bank_stock",
            "description": "Get current stock price and financial data for Indian banks. Use for HDFC Bank, ICICI Bank, or State Bank of India (SBI) stock queries.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Stock ticker symbol: HDFCBANK.NS (HDFC Bank), ICICIBANK.NS (ICICI Bank), or SBIN.NS (SBI)",
                        "enum": ["HDFCBANK.NS", "ICICIBANK.NS", "SBIN.NS"]
                    }
                },
                "required": ["ticker"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for real-time information. Use for gold prices, currency exchange rates, news, weather, or any current events.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }
]

print("Tools registered:")
for tool in TOOLS:
    print(f"  - {tool['function']['name']}: {tool['function']['description'][:60]}...")

Tools registered:
  - indian_bank_stock: Get current stock price and financial data for Indian banks....
  - web_search: Search the web for real-time information. Use for gold price...


## Tool Execution Functions

These functions actually execute the tools when the LLM requests them.

In [6]:
def execute_indian_bank_stock(ticker: str) -> dict:
    """
    Execute the indian_bank_stock tool using Yahoo Finance API.
    """
    print(f"  📈 Fetching {ticker} from Yahoo Finance...")
    
    try:
        stock = yf.Ticker(ticker)
        info = stock.info
        
        current_price = info.get('currentPrice') or info.get('regularMarketPrice')
        previous_close = info.get('previousClose')
        
        result = {
            "ticker": ticker,
            "company": info.get('longName'),
            "current_price": current_price,
            "currency": "INR",
            "previous_close": previous_close,
            "day_change": round(current_price - previous_close, 2) if current_price and previous_close else None,
            "market_cap": info.get('marketCap'),
            "volume": info.get('volume')
        }
        
        print(f"  ✅ Got price: ₹{result['current_price']}")
        return result
        
    except Exception as e:
        return {"error": str(e), "ticker": ticker}


def execute_web_search(query: str) -> str:
    """
    Execute web search using LlamaStack's built-in Tavily integration.
    We use the Responses API which handles web_search server-side.
    """
    print(f"  🌐 Searching web for: {query}")
    
    response = client.responses.create(
        model=model_id,
        input=query,
        tools=[{"type": "web_search"}],
        stream=False
    )
    
    result = response.output_text
    print(f"  ✅ Got web search result")
    return result

## Multi-Tool Agent

The agent uses a simple loop:
1. Send query to LLM with all tool definitions
2. If LLM requests a tool → execute it and send result back
3. LLM generates final response using the tool output

In [8]:
def run_agent(query: str, verbose: bool = True) -> str:
    """
    Run the multi-tool agent. The LLM automatically chooses which tool to use.
    
    Args:
        query: User's question
        verbose: Print debug information
    
    Returns:
        The agent's final response
    """
    if verbose:
        print(f"\n{'='*60}")
        print(f"🤖 Query: {query}")
        print(f"{'='*60}")
    
    # Detailed system message with explicit instructions
    messages = [
        {
            "role": "system",
            "content": """You are a helpful financial assistant. You MUST use tools to answer questions - never make up data.

AVAILABLE TOOLS:
1. indian_bank_stock - Use this for stock prices of:
   - HDFC Bank → ticker: "HDFCBANK.NS"
   - ICICI Bank → ticker: "ICICIBANK.NS"  
   - State Bank of India (SBI) → ticker: "SBIN.NS"

2. web_search - Use this for:
   - Gold prices
   - Currency exchange rates (USD/INR, EUR/INR, etc.)
   - Any news or current events
   - Any other real-time information

INSTRUCTIONS:
- When asked about HDFC, ICICI, or SBI stock prices, call indian_bank_stock with the correct ticker
- When asked about gold, currencies, or news, call web_search with a clear query
- ALWAYS call a tool - never guess or make up prices
- After receiving tool results, summarize them clearly for the user"""
        },
        {"role": "user", "content": query}
    ]
    
    # Step 1: Ask LLM which tool to use
    if verbose:
        print(f"\n📤 Sending request to LLM with {len(TOOLS)} tools...")
    
    response = client.chat.completions.create(
        model=model_id,
        messages=messages,
        tools=TOOLS,
        tool_choice="required"  # Force tool use
    )
    
    # DEBUG: Print the RAW response
    if verbose:
        print(f"\n{'='*60}")
        print(f"🔍 DEBUG - RAW API RESPONSE:")
        print(f"{'='*60}")
        print(f"Response type: {type(response)}")
        print(f"Response object: {response}")
        print(f"\nChoices[0]:")
        print(f"  finish_reason: {response.choices[0].finish_reason}")
        print(f"  message type: {type(response.choices[0].message)}")
        print(f"  message: {response.choices[0].message}")
        print(f"{'='*60}")
    
    assistant_message = response.choices[0].message
    
    # DEBUG: Inspect message attributes
    if verbose:
        print(f"\n🔍 DEBUG - Message attributes:")
        for attr in dir(assistant_message):
            if not attr.startswith('_'):
                try:
                    val = getattr(assistant_message, attr)
                    if not callable(val):
                        print(f"   {attr}: {val}")
                except:
                    pass
    
    # Step 2: Check if LLM wants to call a tool
    if hasattr(assistant_message, 'tool_calls') and assistant_message.tool_calls:
        tool_call = assistant_message.tool_calls[0]
        
        if verbose:
            print(f"\n✅ Tool call detected!")
            print(f"   tool_call object: {tool_call}")
            print(f"   function.name: {tool_call.function.name}")
            print(f"   function.arguments: {tool_call.function.arguments}")
        
        func_name = tool_call.function.name
        func_args = json.loads(tool_call.function.arguments)
        
        print(f"\n🔧 LLM chose tool: {func_name}")
        print(f"   Arguments: {func_args}")
        
        # Step 3: Execute the tool
        if func_name == "indian_bank_stock":
            ticker = func_args.get("ticker", "HDFCBANK.NS")
            print(f"\n🚀 EXECUTING indian_bank_stock with ticker={ticker}")
            tool_result = execute_indian_bank_stock(ticker)
            tool_result_str = json.dumps(tool_result)
            print(f"\n📊 RAW TOOL RESULT: {tool_result_str}")
        elif func_name == "web_search":
            search_query = func_args.get("query", query)
            print(f"\n🚀 EXECUTING web_search with query={search_query}")
            return execute_web_search(search_query)
        else:
            tool_result_str = json.dumps({"error": f"Unknown tool: {func_name}"})
        
        # Step 4: Send tool result back to LLM for final response
        messages.append({
            "role": "assistant",
            "content": None,
            "tool_calls": [tool_call]
        })
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "content": tool_result_str
        })
        
        print(f"\n📤 Sending tool result back to LLM for summary...")
        
        final_response = client.chat.completions.create(
            model=model_id,
            messages=messages
        )
        
        answer = final_response.choices[0].message.content
        print(f"\n✅ Final Answer: {answer}")
    else:
        # No tool call - LLM answered directly (shouldn't happen with tool_choice="required")
        print(f"\n⚠️ WARNING: No tool_calls detected! LLM answered directly.")
        answer = assistant_message.content
        print(f"\n❌ Direct answer (likely hallucinated): {answer}")
    
    return answer

## Test: Stock Price Query

The LLM should automatically choose `indian_bank_stock` tool.

In [9]:
# Test 1: Stock query - should use indian_bank_stock tool
run_agent("What's the current stock price of HDFC Bank?")


🤖 Query: What's the current stock price of HDFC Bank?

📤 Sending request to LLM with 2 tools...


INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)



🔍 DEBUG - RAW API RESPONSE:
Response type: <class 'llama_stack_client.types.chat.completion_create_response.OpenAIChatCompletion'>
Response object: OpenAIChatCompletion(id='chatcmpl-9ce41381c41e47f2adfac154a071ac78', choices=[OpenAIChatCompletionChoice(finish_reason='stop', index=0, message=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content='', name=None, tool_calls=[OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCall(type='function', id='chatcmpl-tool-bca82925f67243ec85f837df310e2bc5', function=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCallFunction(arguments='{"ticker": "HDFCBANK.NS"}', name='indian_bank_stock'), index=None)], refusal=None, annotations=None, audio=None, function_call=None, reasoning_content=None), logprobs=None, stop_reason=None)], created=1767843894, model='granite-3-3-8b-instruct', object='chat.completion', usage=OpenAIChatCompletionUsage(completion_tokens=38, prompt_tokens=565, total_tokens

INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"



✅ Final Answer: The current stock price of HDFC Bank (ticker: HDFCBANK.NS) is approximately 949.05 INR. It's down by 13.15 INR from the previous day's close of 962.2 INR. The market capitalization stands at around 14,60,069,51,85,408 INR, and the volume of shares traded today is approximately 52,88,47,22.


"The current stock price of HDFC Bank (ticker: HDFCBANK.NS) is approximately 949.05 INR. It's down by 13.15 INR from the previous day's close of 962.2 INR. The market capitalization stands at around 14,60,069,51,85,408 INR, and the volume of shares traded today is approximately 52,88,47,22."

In [10]:
# Test 2: Another stock query
run_agent("What is SBI's stock price today?")


🤖 Query: What is SBI's stock price today?

📤 Sending request to LLM with 2 tools...


INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)



🔍 DEBUG - RAW API RESPONSE:
Response type: <class 'llama_stack_client.types.chat.completion_create_response.OpenAIChatCompletion'>
Response object: OpenAIChatCompletion(id='chatcmpl-3644bae8679d4301bdf66148b2b07d34', choices=[OpenAIChatCompletionChoice(finish_reason='stop', index=0, message=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content='', name=None, tool_calls=[OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCall(type='function', id='chatcmpl-tool-69aca906639f45f2ba0574f7e481f90f', function=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCallFunction(arguments='{"ticker": "SBIN.NS"}', name='indian_bank_stock'), index=None)], refusal=None, annotations=None, audio=None, function_call=None, reasoning_content=None), logprobs=None, stop_reason=None)], created=1767843955, model='granite-3-3-8b-instruct', object='chat.completion', usage=OpenAIChatCompletionUsage(completion_tokens=37, prompt_tokens=563, total_tokens=600

INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"



✅ Final Answer: The current stock price of State Bank of India (SBI) is 1006.4 INR. The previous close was 1007.15 INR, indicating a day change of -0.75 INR. The market cap stands at approximately 92,89,69,39,69,920 INR and the volume of shares traded is 22,61,710.


'The current stock price of State Bank of India (SBI) is 1006.4 INR. The previous close was 1007.15 INR, indicating a day change of -0.75 INR. The market cap stands at approximately 92,89,69,39,69,920 INR and the volume of shares traded is 22,61,710.'

## Test: Web Search Query

The LLM should automatically choose `web_search` tool.

In [11]:
# Test 3: Web search - should use web_search tool
run_agent("What is the current gold price per ounce?")


🤖 Query: What is the current gold price per ounce?

📤 Sending request to LLM with 2 tools...


INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)



🔍 DEBUG - RAW API RESPONSE:
Response type: <class 'llama_stack_client.types.chat.completion_create_response.OpenAIChatCompletion'>
Response object: OpenAIChatCompletion(id='chatcmpl-da3e830c6d8b4b26b07e090a11369359', choices=[OpenAIChatCompletionChoice(finish_reason='stop', index=0, message=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content='', name=None, tool_calls=[OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCall(type='function', id='chatcmpl-tool-32933d24f2314b5ea2932788635a4921', function=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCallFunction(arguments='{"query": "current gold price per ounce"}', name='web_search'), index=None)], refusal=None, annotations=None, audio=None, function_call=None, reasoning_content=None), logprobs=None, stop_reason=None)], created=1767843991, model='granite-3-3-8b-instruct', object='chat.completion', usage=OpenAIChatCompletionUsage(completion_tokens=27, prompt_tokens=564, tot

INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/responses "HTTP/1.1 200 OK"


  ✅ Got web search result


"According to the recent searches, the current price of gold per ounce is around $4,500 USD. This value can slightly vary as the gold market frequently fluctuates. You can find more detailed and up-to-the-minute prices on these sites:\n\n1. [Gold Price per Ounce](https://goldprice.org/gold-price.html): Got current gold prices including live gold price charts and historical gold price graphs.\n  \n2. [Live Gold Spot Price Chart](https://www.bullionvault.com/gold-price-chart.do): Offers continuous tracking of the real-time changing gold price.\n\n3. [Live gold price today in USD](https://goldavenue.com/en/gold-price/usd/1-oz?srsltid=AfmBOop9m8MgnwUyQXMp2yjWAixt-g-jMVGbljMNmWLQ779b7fKHmJDA): Directly shows the price notice for 1 ounce of gold in US Dollars.\n\n4. [Gold Spot Price Chart | Markets Insider](https://markets.businessinsider.com/commodities/gold-price): Indicates current gold price at $4,490.94 for one ounce in USD with a trend line over recent values.\n\n5. [Trading Economics]

In [12]:
# Test 4: Currency exchange rate
run_agent("What is the USD to INR exchange rate today?")


🤖 Query: What is the USD to INR exchange rate today?

📤 Sending request to LLM with 2 tools...


INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/chat/completions "HTTP/1.1 200 OK"
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_computed_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)
/var/folders/cr/z28pm9ps0692ghh99t3d2zpr0000gn/T/ipykernel_48816/3529673231.py:76: PydanticDeprecatedSince211: Accessing the 'model_fields' attribute on the instance is deprecated. Instead, you should access this attribute from the model class. Deprecated in Pydantic V2.11 to be removed in V3.0.
  val = getattr(assistant_message, attr)



🔍 DEBUG - RAW API RESPONSE:
Response type: <class 'llama_stack_client.types.chat.completion_create_response.OpenAIChatCompletion'>
Response object: OpenAIChatCompletion(id='chatcmpl-bab48ac90b3d4237904e3a614fa18ee3', choices=[OpenAIChatCompletionChoice(finish_reason='stop', index=0, message=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParam(role='assistant', content='', name=None, tool_calls=[OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCall(type='function', id='chatcmpl-tool-d5deec7e8cad42c7859ccaecb47032cf', function=OpenAIChatCompletionChoiceMessageOpenAIAssistantMessageParamToolCallFunction(arguments='{"query": "USD to INR exchange rate today"}', name='web_search'), index=None)], refusal=None, annotations=None, audio=None, function_call=None, reasoning_content=None), logprobs=None, stop_reason=None)], created=1767844017, model='granite-3-3-8b-instruct', object='chat.completion', usage=OpenAIChatCompletionUsage(completion_tokens=25, prompt_tokens=565, t

INFO:httpx:HTTP Request: POST https://llama-stack-ext-competitor-analysis.apps.ocp.sx7qw.sandbox2219.opentlc.com/v1/responses "HTTP/1.1 200 OK"


  ✅ Got web search result


"According to the latest web data, as of today, the USD to INR exchange rate is around INR 84.27 to INR 90.12 for 1 US dollar. These rates may vary slightly between currency conversion services. Please visit the sources below for the most current rates:\n\n1. [USD to INR - Today's Best US Dollar to Rupee Exchange Rate](https://www.compareremit.com/todays-best-dollar-to-rupee-exchange-rate/)\n2. [US dollars to Indian rupees Exchange Rate](https://wise.com/us/currency-converter/usd-to-inr-rate)\n3. [US Dollar Rate Today | Live USD Buying & Selling Rate](https://www.bookmyforex.com/us-dollar/rates/)\n4. [1 USD to INR - US Dollars to Indian Rupees Exchange Rate - Xe](https://xe.com/en-us/currencyconverter/convert/?Amount=1&From=USD&To=INR)\n5. [USD to INR Exchange Rates - Convert US dollars to Indian rupees](https://www.remitly.com/us/en/currency-converter/usd-to-inr-rate)\n\nPlease note that exchange rates fluctuate continuously throughout the trading day, so for the real-time rate, pleas

## Summary

### Key Points for LlamaStack 0.3.0 Multi-Tool Agents:

1. **Use Chat Completions API** with `tools` parameter for tool definitions
2. **Define tools with OpenAI-compatible function schemas** (type, function, name, description, parameters)
3. **Let the LLM decide** which tool to use via `tool_choice="auto"`
4. **Execute tools client-side** when the LLM requests them
5. **For server-side tools** like `web_search`, use the Responses API
6. **Return tool results** to the LLM to generate the final response

### API Patterns:
```python
# Client-side tools (Yahoo Finance, custom APIs):
client.chat.completions.create(model, messages, tools=[...], tool_choice="auto")

# Server-side tools (web_search):
client.responses.create(model, input, tools=[{"type": "web_search"}])
```